In [1]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gym import Env
from gym import spaces
import random
from IPython.display import clear_output
import os
import matplotlib
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv
import seaborn as sns

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = "cpu"

import environment
from seq_buffer import Transition, SequentialReplayBuffer

In [2]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)], [8 * math.pow(10, 9), 8.5* math.pow(10, 9)], [6 * math.pow(10, 9), 6.2* math.pow(10, 9)]]

hop_states = []

for i in range (0,3):
    radar_states = []
    interval = (freq_agile_radar_versions[i][1] - freq_agile_radar_versions[i][0])/10
    for j in range(0,24):
        radar_states.append(int(freq_agile_radar_versions[i][0]+ interval * j))
    hop_states.append(radar_states)

jamming_bandwidths = [10e6, 25e6, 40e6]

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import namedtuple, deque
from copy import deepcopy

class DRQNWithCommunication(nn.Module):
    def __init__(self, input_dim, hidden_dim, action_dim, message_dim=32):
        super(DRQNWithCommunication, self).__init__()
        self.hidden_dim = hidden_dim
        self.message_dim = message_dim

        # Observation encoder
        self.obs_encoder = nn.Linear(input_dim, hidden_dim)

        # RNN for temporal processing
        self.rnn = nn.LSTM(hidden_dim + message_dim, hidden_dim, batch_first=True)

        # Action-value output
        self.q_net = nn.Linear(hidden_dim, action_dim)

        # Message output
        self.message_net = nn.Linear(hidden_dim, message_dim)

    def forward(self, obs, hidden_state, message=None):
        """
        Forward pass for DRQN with communication.
        Args:
            obs: Observations (batch_size x input_dim).
            hidden_state: Hidden state for the RNN.
            message: Incoming message (batch_size x message_dim).
        """
        batch_size = obs.size(0)

        # Encode observation
        encoded_obs = self.obs_encoder(obs)  # Shape: (batch_size, hidden_dim)

        # Use default message if none provided
        if message is None:
            message = torch.zeros(batch_size, self.message_dim, device=obs.device)  # Shape: (batch_size, message_dim)
        if message.dim() == 3:
            message = message.squeeze(1)

        # Concatenate observation encoding and message
        rnn_input = torch.cat([encoded_obs, message], dim=-1)  # Shape: (batch_size, hidden_dim + message_dim)

        # RNN processing
        rnn_output, hidden_state = self.rnn(rnn_input.unsqueeze(1), hidden_state)  # RNN expects 3D input: (batch_size, seq_len, input_dim)
        rnn_output = rnn_output.squeeze(1)  # Shape: (batch_size, hidden_dim)

        # Compute Q-values and outgoing message
        q_values = self.q_net(rnn_output)  # Shape: (batch_size, action_dim)
        outgoing_message = self.message_net(rnn_output)  # Shape: (batch_size, message_dim)

        return q_values, outgoing_message, hidden_state

    def init_hidden(self, batch_size, device):
        """Initialize the hidden state for the RNN."""
        return (torch.zeros(1, batch_size, self.hidden_dim, device=device),
                torch.zeros(1, batch_size, self.hidden_dim, device=device))

In [4]:
class CommunicationAgent:
    def __init__(self, input_dim, hidden_dim, action_dim, message_dim, lr=1e-4, gamma=0.90, device="cpu"):
        self.device = device
        self.gamma = gamma

        # DRQN with communication
        self.model = DRQNWithCommunication(input_dim, hidden_dim, action_dim, message_dim).to(device)
        self.target_model = deepcopy(self.model)

        # Optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

        # Replay buffer
        self.memory = deque(maxlen=10000)

    def act(self, state, hidden_state, message, epsilon):
        """
        Select an action using epsilon-greedy policy.
        Args:
        state: The current observation (state) for the agent.
        hidden_state: The hidden state for the RNN.
        message: The incoming communication message (None if no message).
        epsilon: Probability of taking a random action.
        Returns:
        action: Selected action.
        outgoing_message: The message generated for communication.
        """
        # Use default message if none is provided
        if message is None:
            message = torch.zeros(self.model.message_dim, device=self.device)  # Correctly initialize default message

        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            return np.random.randint(0, self.model.q_net.out_features), np.zeros(self.model.message_dim)

        # Prepare inputs
        state_tensor = torch.FloatTensor(state).to(self.device).unsqueeze(0)
        message_tensor = torch.FloatTensor(message).to(self.device).unsqueeze(0)

        # Get Q-values and outgoing message
        q_values, outgoing_message, _ = self.model(state_tensor, hidden_state, message_tensor)

        # Select action
        action = q_values.argmax().item()

        return action, outgoing_message.detach().cpu().numpy()

    def remember(self, *args):
        """Store experiences in the replay buffer."""
        self.memory.append(Transition(*args))

    def replay(self, batch_size, sequence_length=10):
        if len(self.memory) < batch_size:
            return

        # Sample sequences of transitions
        batch = random.sample(self.memory, batch_size)
        transitions = Transition(*zip(*batch))

        # Convert the sampled transitions into tensors
        states = torch.FloatTensor(transitions.state).to(self.device)
        actions = torch.LongTensor(transitions.action).to(self.device)
        rewards = torch.FloatTensor(transitions.reward).to(self.device)
        next_states = torch.FloatTensor(transitions.next_state).to(self.device)

        # Compute target Q-values
        with torch.no_grad():
            target_q_values, _, _ = self.target_model(next_states, self.model.init_hidden(batch_size, self.device))
            max_next_q_values = target_q_values.max(dim=1)[0]
            targets = rewards + self.gamma * max_next_q_values

        # Compute predicted Q-values
        predicted_q_values, _, _ = self.model(states, self.model.init_hidden(batch_size, self.device))
        predicted_q_values = predicted_q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        # Loss and optimization
        loss = nn.MSELoss()(predicted_q_values, targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def update_target_model(self):
        """Update target model weights."""
        self.target_model.load_state_dict(self.model.state_dict())

In [ ]:
env = environment.ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

# Define agents
agents = {name: CommunicationAgent(input_dim=sum(env.n_frequencies), hidden_dim=256, 
                                   action_dim=env.action_space[name].n, 
                                   message_dim=32, device=device) for name in env.agents}

messages = {agent: np.zeros(agents[agent].model.message_dim) for agent in env.agents}
hidden_states = {agent: agents[agent].model.init_hidden(1, device="cpu") for agent in env.agents}
epsilon = 1.0

episodes = 8000

for episode in range(1, episodes + 1):
    state = env.reset()
    total_reward = 0
    done = {agent: False for agent in env.agents}

    while not all(done.values()):
        actions = {}
        for agent in env.agents:
            actions[agent], messages[agent] = agents[agent].act(
                state[agent], hidden_states[agent], messages[agent], epsilon
            )

        next_state, rewards, done, _ = env.step(actions)
        total_reward += sum(rewards.values())

        # Store transitions for each agent
        for agent in env.agents:
            agents[agent].remember(state[agent], actions[agent], next_state[agent], rewards[agent])

        state = next_state

    epsilon = max(0.10, epsilon * 0.995)  # Decay exploration rate

    # Training
    for agent in agents:
        agents[agent].replay(batch_size=16)
        if episode % 20 == 0:
            agents[agent].update_target_model()

    print(f"Episode {episode}, Total Reward: {total_reward}")

C:\Users\Abhim\AppData\Local\Temp\ipykernel_15784\3656154687.py:61: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  states = torch.FloatTensor(transitions.state).to(self.device)


Episode 1, Total Reward: -41400
Episode 2, Total Reward: -42640
Episode 3, Total Reward: -35760
Episode 4, Total Reward: -51760
Episode 5, Total Reward: -41450
Episode 6, Total Reward: -46200
Episode 7, Total Reward: -39350
Episode 8, Total Reward: -52810
Episode 9, Total Reward: -27840
Episode 10, Total Reward: -53100
Episode 11, Total Reward: -45670
Episode 12, Total Reward: -26170
Episode 13, Total Reward: -43100
Episode 14, Total Reward: -48350
Episode 15, Total Reward: -34700
Episode 16, Total Reward: -54270
Episode 17, Total Reward: -46980
Episode 18, Total Reward: -43220
Episode 19, Total Reward: -37990
Episode 20, Total Reward: -42900
Episode 21, Total Reward: -44920
Episode 22, Total Reward: -54410
Episode 23, Total Reward: -31270
Episode 24, Total Reward: -43580
Episode 25, Total Reward: -44380
Episode 26, Total Reward: -47700
Episode 27, Total Reward: -52740
Episode 28, Total Reward: -47560
Episode 29, Total Reward: -52820
Episode 30, Total Reward: -49200
Episode 31, Total R